In [1]:
import pickle
project = 'qt'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data = pickle.load(f_train)
        te_data = pickle.load(f_test)
pred_list = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list.append(pickle.load(f))
tr_pred = [[pred_list[0][j], pred_list[1][j], pred_list[2][j]] for j in range(len(pred_list[0]))]
te_pred = [[pred_list[3][j], pred_list[4][j], pred_list[5][j]] for j in range(len(pred_list[3]))]                
tr_data = [tr_pred, tr_data[5]]
te_data = [te_pred, te_data[5]]

In [26]:
pred_list[0]

array([0.07693368, 0.35020531, 0.03905198, ..., 0.18881377, 0.05588778,
       0.09425423])

In [2]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

# 仮想的なモデルの出力と真の値（目標値）の設定
probs_model1 = pred_list[0]
probs_model2= pred_list[1]
probs_model3 = pred_list[2]
true_labels = tr_data[1]

# 目的関数の定義（auc）
def objective(weights):
    print(weights)
    weighted_probs = weights[0] * probs_model1 + weights[1] * probs_model2 + weights[2] * probs_model3
    auc = roc_auc_score(true_labels, weighted_probs)
    print(auc)
    return -auc 

# 初期の重みの設定
initial_weights = np.ones(3) / 3  # 3つのモデルのために均等な重みで初期化

# 制約条件
constraints = ({'type': 'ineq', 'fun': lambda x: x},{'type': 'ineq', 'fun': lambda w: np.sum(w) - 1})  # 重みの合計は1になるように制約

# 最適化の実行
result = minimize(objective, initial_weights, constraints=constraints, method='COBYLA')

# 結果の表示
optimal_weights = result.x
print("Optimal Weights:", optimal_weights)


[0.33333333 0.33333333 0.33333333]
0.8119628528480407
[1.33333333 0.33333333 0.33333333]
0.7965049284096011
[0.33333333 1.33333333 0.33333333]
0.8093564669412117
[0.33333333 0.33333333 1.33333333]
0.7769672161541041
[0. 1. 0.]
0.7913915404406081
[0.29587585 0.70412415 0.        ]
0.8049319597045813
[0.51407637 0.16080973 0.3251139 ]
0.800372989570046
[0.26116455 0.26116455 0.26116455]
0.8119628528480407
[0.13875095 0.3772085  0.48404055]
0.803747330411903
[0.33509254 0.42082895 0.24407851]
0.8137328953202393
[0.42807764 0.41077577 0.16114659]
0.8118944915731593
[0.24914199 0.51146843 0.23938958]
0.8131576914111613
[0.29211726 0.46614869 0.24173405]
0.8137027031044513
[0.35313474 0.43887114 0.26212071]
0.8137342837810193
[0.32687805 0.4142856  0.31323223]
0.8134226151729578
[0.36905509 0.44852668 0.23702339]
0.8136005287258651
[0.36316741 0.4270299  0.26392928]
0.8136600419663632
[0.32780326 0.45023483 0.27646453]
0.8137789323237534
[0.31989335 0.47536854 0.25966298]
0.8137684508060994


In [3]:
weighted_average2 = optimal_weights[0] * pred_list[3] + optimal_weights[1] * pred_list[4] + optimal_weights[2] * pred_list[5]
roc_auc_score(te_data[1] ,weighted_average2)

0.772596260438992

In [1]:
import numpy as np
from scipy.optimize import minimize

# 仮想的なモデルの出力と真の値（目標値）の設定
model1_output = np.array([1.0, 2.0, 3.0])
model2_output = np.array([1.5, 2.5, 3.5])
model3_output = np.array([0.5, 1.5, 2.5])
true_values = np.array([2.0, 3.0, 4.0])

# 目的関数の定義（平均二乗誤差）
def objective(weights):
    weighted_average = weights[0] * model1_output + weights[1] * model2_output + weights[2] * model3_output
    mse = np.mean((weighted_average - true_values)**2)
    return mse

# 初期の重みの設定
initial_weights = np.array([0.33, 0.33, 0.33])  # 初期の重みの総和は1.0になるように設定

# 最適化の実行
result = minimize(objective, initial_weights, method='SLSQP', bounds=[(0, 1)] * 3, constraints={'type': 'eq', 'fun': lambda w: 1.0 - np.sum(w)})

# 結果の表示
optimal_weights = result.x
print("Optimal Weights:", optimal_weights)


Optimal Weights: [0.00000000e+00 1.00000000e+00 1.66533454e-16]


In [11]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

# 3つのモデルの確率値
probs_model1 = np.array([0.2, 0.4, 0.6, 0.8])
probs_model2 = np.array([0.3, 0.5, 0.7, 0.9])
probs_model3 = np.array([0.1, 0.3, 0.5, 0.7])

# 真のラベル
true_labels = np.array([0, 1, 0, 1])

# 目的関数の定義
def objective(weights, probs_list, true_labels):
    weighted_probs = np.dot(weights, probs_list)
    auc = roc_auc_score(true_labels, weighted_probs)
    return -auc  # 最大化するためにマイナスをかける

# 初期の重みの設定
initial_weights = np.ones(3) / 3  # 3つのモデルのために均等な重みで初期化

# 制約条件
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})  # 重みの合計は1になるように制約

# 最適化の実行
result = minimize(objective, initial_weights, args=(probs_model1, probs_model2, probs_model3, true_labels),
                  constraints=constraints)

# 最適な重み
optimal_weights = result.x
print("Optimal Weights:", optimal_weights)

# 最適な重みで計算された確率値
optimal_weighted_probs = np.dot(optimal_weights, [probs_model1, probs_model2, probs_model3])

# 最適な重みで計算されたAUC
optimal_auc = roc_auc_score(true_labels, optimal_weighted_probs)
print("Optimal AUC:", optimal_auc)


TypeError: objective() takes 3 positional arguments but 5 were given